In [ ]:
import numpy as np

# PyBeast Tuorial 2: Adding Sensors to Agents 

In this tutorial, you will learn how to use sensors in BEAST to enable agents to autonomously interact with their environment. By default, any agent that inherits from the `core.agents.animat.Animat` can be equipped with sensors. BEAST provides a variety of predefined sensors that can be instantiated using the functions provided in the `core.sensors.sensor` module. User can create their own customized sensors by inherting from the ```core.sensors.sensorbase.Sensor``` base class. 

In BEAST, sensors are type sensitive, i.e. they only detect agents or objects of a desired type. Specifically, the `Sensor` class has a `MatchFunc` attribute, a callable responsible for type checking.  Whenever an agent or object of the desired type is detected, the ```Sensor.EvalFunc``` is called, which updates the sensor's internal state. At any time, a sensor's output can be retrieved using the ```Sensor.GetOutput``` method. The sensor's output is filtered by the ```Sensor.ScalFunc``` attribute, which is a callable that scales the output to a desired value range, typically from -1 to 1.      

Two useful sensors available in BEAST are the proximity sensor and the nearest angle sensor. You will use these sensors to create two simple experiments featuring autonomously behaving agents. 

## Proximity sensor 

The proximity sensor functions as a beam sensor that detects objects or agents in given scope and range. Let's use the skeleton of the ```BraitenbergVehicle``` class introduced in the first tutorial and add two proximity sensors into its design.    

In [ ]:
from pybeast.core.agents.animat import Animat
from pybeast.core.sensors.sensor import ProximitySensor

class BraitenbergVehicle(Animat):

    def __init__(self):
    
        super().__init__()

        leftSensor = ProximitySensor(BraitenbergVehicle, scope = np.pi/3.5, range = 200.0, orientation=np.pi/6, simple=True)
        rightSensor = ProximitySensor(BraitenbergVehicle, scope = np.pi/3.5, range = 200.0, orientation=-np.pi/6, simple=True) 
        
        self.AddSensor('left', leftSensor)
        self.AddSensor('right', rightSensor)

        self.SetInteractionRange(200.0)

        self.SetSolid(False)
        self.SetMaxSpeed(100.0)
        self.SetMinSpeed(0.0)
        self.SetRadius(10.0)
        self.SetMaxRotationSpeed(2*np.pi)

    
    def Control(self):

        lOutput = self.sensors['left'].GetOutput()
        rOutput = self.sensors['right'].GetOutput()
        
        self.controls["left"]  = 1.0 - lOutput
        self.controls["right"] = 1.0 - rOutput


In `BraitenbergVehicle.__init__` method, we call the `ProximitySensor` function to create two beam sensors. As the first argument, we pass a reference to `BraitenbergVehicle` class, i.e. the sensor detect other Braitenberg vehicles. The scope of each sensor is set to $\pi/4$ (45 degrees), and the range is set to 200. Note that sensor range should be defined relative to the world's dimensions, which bey defauly are 800 width and 600 height. The `leftSensor` 'is oriented relative to vehicle's orientation at $+\pi/8$ (25 degrees), i.e. it covers the vehicle's left field of vision. The ```rightSensor``` is oriented realtive to vehicle's orientation at $-\pi/8$, i.e. it covers the vehicle's right field of vision. After instantiation, the left and right sensor are added to the vehicle using the ```Animat.AddSensor``` method. To better understand how sensors work in BEAST, let's create shiny new Braitenberg vehicle with sensors        

In [ ]:
braiti = BraitenbergVehicle()
braiti.Init()

The sensors of an agent that inherits from the ```Animat``` class can be accessed via the ```Animat.sensors``` attribute 

In [ ]:
braiti.sensors

which is a dicitonary of name sensor pairs. The output of a sensor can be retrieved using the ```Sensor.GetOutput``` method as already mentioned in the introduction

In [ ]:
braiti.sensors['left'].GetOutput(), braiti.sensors['right'].GetOutput() 

The output of both sensors is currently zero because they have not detected anything yet. To test the sensors, we need to create a simulation environment and add instances of the `BraitenbergVehicle` to its world. To control the movement of these vehicles during the simulation, we must implement the vehicle's `Control` method as discussed in the previous tutorial.

To enable the `BraitenbergVehicle` to respond autonomously to other Braitenberg vehicles, we need to adjust the values in its `controls` dictionary based on the sensor outputs. To demonstrate this, we have implemented a simple control method within `BraitenbergVehicle.Control`. First, we retrieve the outputs of the left and right proximity sensors using their `GetOutput` method. By default, the ProximitySensor measures the distance to the nearest object within its detection scope and range and that matches the desired type. It then linearly scales this distance to a value between 1.0 and 0.0, i.e. closer object yield higher outputs. 

If both sensor outputs are zero, indicating no vehicles were detected within range, the vehicle’s left and right control values are set to 1.0, causing the vehicle to move in a straight line at maximum speed. However, if either sensor detects a vehicle (indicated by a non-zero output), the corresponding control values are decreased. Can you guess what behavior this will produce? Let's create a simulation with multiple Braitenberg vehicles to test our hypothesis.

In [ ]:
from pybeast.core.simulation import Simulation

class BraitenbergSimulation(Simulation):

    def __init__(self):
        
        super().__init__('Breitenberg')

        self.numberVehicles = 5
        
        self.Runs = 1
        self.generations = 1
        self.assessments = 1
        self.timeSteps = -1
        
        self.whatToLog['Generation'] = self.whatToLog['Assessment'] = True  
         
    def BeginAssessment(self):

        for _ in range(self.numberVehicles):
        
            self.theWorld.Add(BraitenbergVehicle())
        
        super().BeginAssessment()


Setting the ```BraitenbergSimulation.timeSteps``` to -1 makes the assessment run forever.  Let's run the simulation in the BEAST GUI and see how our vehicle's behave.  

In [ ]:
simulation = BraitenbergSimulation()
simulation.RunSimulation(render=True)

## Nearest angle sensor

The nearest angle sensor detects the relative angle to the closest agent or object that matches a specified type. It functions like a beam sensor with 360-degree scope. To demonstrate the nearest angle sensor in action, we define a `Mouse` class, modeled as a Braitenberg vehicle, that can detect and eat cheese objects. We can create custom inanimate objects by creating a new class that inherits from the `core.world.worldobject.WorldObject` base class.

In [ ]:
from pybeast.core.world.worldobject import WorldObject
from pybeast.core.sensors.sensor import NearestAngleSensor
from pybeast.core.utils.colours import ColourPalette, ColourType

class Cheese(WorldObject):
    """Represents a cheese object."""

    def __init__(self):
        """Initialize a new Cheese object."""
        super().__init__()
        self.SetRadius(10.0)
        self.SetColour(*ColourPalette[ColourType.COLOUR_YELLOW])

    def Eaten(self):
        """Handle the event when the cheese is eaten."""
        self.SetDead(True)
    

By default, world objects are initialized with a random position. Unlike the `Animat` class, the `WorldObject` does not implement a default `Update` method, meaning these objects remain stationary unless you explicitly define movement by implementing their `Update` method. The constructor `Cheese.__init__` sets the cheese's radius to 10.0 and its color to yellow. The `Cheese.Eaten` method is called from within the `Mouse` class whenever a collision with a cheese object occurs. This method sets the `dead` attribute to `True`, preventing the object from being displayed. With the `Cheese` class implemented, let's move on to defining the `Mouse` class."

In [ ]:
class Mouse(Animat):

    def __init__(self, numberCheese):
    
        super().__init__()

        self.numberCheese = numberCheese
        
        sensorRange = 400.0
        angleSensor = NearestAngleSensor(Cheese, range = sensorRange)
        
        self.AddSensor('angle', angleSensor)        
        self.SetInteractionRange(sensorRange)

        self.SetSolid(False)
        self.SetMaxSpeed(100.0)
        self.SetMinSpeed(0.0)
        self.SetRadius(10.0)

        self.cheeseEaten = 0
    
    def Control(self):
        
        o = self.sensors['angle'].GetOutput()

        if self.cheeseEaten < self.numberCheese:    
            self.controls["left"] = 0.5 + 0.5*o 
            self.controls["right"] = 0.5 - 0.5*o    
        else:
            self.controls["left"] = 0.0
            self.controls["right"] = 0.0

    def OnCollision(self, other):

        if type(other) == Cheese:
            self.myWorld.mySimulation.logger.info('Yammi, cheese!')
            other.Eaten()
            self.cheeseEaten += 1 

            if self.cheeseEaten == self.numberCheese:
                self.myWorld.mySimulation.logger.info('I eaten all the cheese. Time for a nap!')

For the `Mouse` class, we followed a similar template to the `BraitenbergVehicle` class. The constructor `Mouse.__init__` instantiates a nearest angle sensor that detects objects of type `Cheese` and adds it to the mouse. The nearest angle sensor outputs the relative angle of the closest object of the specified type within its detection range. Objects in the sensor's left field of vision correspond to angles between 0 and 180 degrees, while objects in the right field of vision have angles ranging from 0 to -180 degrees. The sensor's `ScalFunc` scales these angles to a value range from [-1, 1]. 

To enable our mouse to eat the cheese, we need to implement the `Animat.OnCollision` method, which is called whenever an animat collides with another object or agent in the world. In `Mouse.OnCollision`, we first check if the object or agent the mouse collided with is of type `Cheese`. If it is, we call the `Cheese.Eaten` method and increase the mouse's count of eaten cheese by one.

Based on what you've learned so far, take a look at the `Mouse.Control` method and think about what behaviour it will produce. Let's simulate the `Mouse` class to test your hypothesis. 

In [ ]:
from pybeast.core.evolve.population import Group

class MouseSimulation(Simulation):
    """Represents a simulation with mice and cheese."""

    def __init__(self):
        """Initialize a new MouseSimulation."""
        super().__init__('Mouse')

        self.generations = 1
        self.assessments = 1
        self.timeSteps = -1
        
        self.numberCheese = 30
        theCheeses = Group(self.numberCheese, Cheese)
        self.Add('thecheese', theCheeses)
        
        self.whatToLog['Generation'] = self.whatToLog['Assessment'] = True

    def BeginAssessment(self):

        mouse = Mouse(self.numberCheese)
        self.theWorld.Add(mouse)
        
        super().BeginAssessment()


In the constructor `MouseSimulation.__init__`, we used `core.evolve.population.Group` class to create a group 30 cheese objects. Groups can be added to the simulation using the `Simulation.Add` method, and they will be automatically added to simulation's `World` object at the beginning of the each assessment. To add the mouse to the world, the have overwritten the `Simulation.BeginAssessment`. Let's start BEAST to check whether our mouse is able to find the cheese.     

In [ ]:
simulation = MouseSimulation()
simulation.RunSimulation(render=True)